In [2]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import gensim
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import strip_punctuation
print((gensim.__version__)) 
import ast
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge

4.1.2


In [3]:
y_test = pd.read_csv("C:/Users/berl03/Desktop/LIME/1 příprava dat/y_test.csv")
y_train = pd.read_csv("C:/Users/berl03/Desktop/LIME/1 příprava dat/y_train.csv")
X_train = pd.read_csv("C:/Users/berl03/Desktop/LIME/1 příprava dat/X_train.csv")
X_test = pd.read_csv("C:/Users/berl03/Desktop/LIME/1 příprava dat/X_test.csv")
X_train_feats = X_train.drop(columns=["Unnamed: 0"])
X_test_feats  = X_test .drop(columns=["Unnamed: 0"])
y_test['abstract_tokenized'] = y_test['abstract_tokenized'].apply(ast.literal_eval)
y_train['abstract_tokenized'] = y_train['abstract_tokenized'].apply(ast.literal_eval)
loaded_model = lreg_w2v_avg = pickle.load(open("C:/Users/berl03/Desktop/LIME/1 příprava dat/lreg_w2v_avg_2020.sav", 'rb'))
w2v_model =gensim.models.Word2Vec.load("C:/Users/berl03/Desktop/LIME/1 příprava dat/w2v_published_only_english_between_2019 and 2020.model")

In [4]:
y_test

Unnamed: 0                            doi  target  \
0              0      10.1186/s13756-019-0662-8       0   
1              1      10.1038/s41541-019-0148-y       0   
2              2      10.1186/s41747-019-0134-1       0   
3              3   10.1371/journal.pone.0226483       0   
4              4   10.1371/journal.pmed.1003018       0   
...          ...                            ...     ...   
8640        8640             10.1111/bjso.12393       0   
8641        8641     10.1038/s41467-020-19057-5       1   
8642        8642   10.1016/j.neuron.2020.06.019       1   
8643        8643  10.1080/07391102.2020.1765876       0   
8644        8644       10.1590/0074-02760200225       0   

                                       abstract_cleaned  \
0     antimicrobial resistance amr compromise treatm...   
1     mycobacterium tuberculosis mtb responsible dea...   
2     wide range cancer immunotherapy approach devel...   
3     modern society exposed myriad risk ranging dis...   
4     co-authors discus use decision analysis planni...   
...                                                 ...   
8640  notion psychological frailty forefront debate ...   
8641  relationship sars-cov-2 viral load risk diseas...   
8642  detrimental impact recreational drug use human...   
8643  reemergence coronavirus prompt need developmen...   
8644  near future overlap coronavirus disease 2019 c...   

                                     abstract_tokenized  
0     [antimicrobial, resistance, amr, compromise, t...  
1     [mycobacterium, tuberculosis, mtb, responsible...  
2     [wide, range, cancer, immunotherapy, approach,...  
3     [modern, society, exposed, myriad, risk, rangi...  
4     [co-authors, discus, use, decision, analysis, ...  
...                                                 ...  
8640  [notion, psychological, frailty, forefront, de...  
8641  [relationship, sars-cov-2, viral, load, risk, ...  
8642  [detrimental, impact, recreational, drug, use,...  
8643  [reemergence, coronavirus, prompt, need, devel...  
8644  [near, future, overlap, coronavirus, disease, ...  

[8645 rows x 5 columns]

In [16]:
docs_train_tok = y_train['abstract_tokenized'].tolist()
docs_test_tok  = y_test ['abstract_tokenized'].tolist()


from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000, analyzer=lambda x: x, token_pattern=None, ngram_range=(1,2))

X_bow_train = vect.fit_transform(docs_train_tok)
X_bow_test  = vect.transform(docs_test_tok)

# --- 4) Vypočti surrogate target na trénovacích datech ------------------
y_sur_train = loaded_model.predict_proba(X_train_feats)[:, 1]
y_sur_test  = loaded_model.predict_proba(X_test_feats )[:, 1]

# --- 5) Trénuj Decision Tree surrogate ----------------------------------
surrogate_tree = DecisionTreeRegressor(
    max_depth=100,    # hloubka 4 je obvykle dostatečně čitelná
    random_state=42
)
surrogate_tree.fit(X_bow_train, y_sur_train)

# --- 6) Feature importances ---------------------------------------------
importances = surrogate_tree.feature_importances_
words       = vect.get_feature_names_out()

df_word_imp = (
    pd.DataFrame({
        "word":       words,
        "importance": importances
    })
    .query("importance > 0")
    .sort_values("importance", ascending=False)
    .reset_index(drop=True)
)

# Normalizovaná důležitost na [0,1]
imp = df_word_imp["importance"]
df_word_imp["importance_norm"] = (imp - imp.min()) / (imp.max() - imp.min())

# --- 7) (volitelně) Vyhodnocení fidelity surrogate ---------------------
from sklearn.metrics import r2_score
fidelity = r2_score(y_sur_test, surrogate_tree.predict(X_bow_test))
print(f"Surrogate fidelity (R²): {fidelity:.3f}")

# Výsledek máš v df_word_imp:
print(df_word_imp.head(20))

C:\Users\berl03\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:539: UserWarning: The parameter 'ngram_range' will not be used since 'analyzer' is callable'
  warnings.warn(
C:\Users\berl03\Anaconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
C:\Users\berl03\Anaconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Surrogate fidelity (R²): 0.892
            word  importance  importance_norm
0    coronavirus    0.237418         1.000000
1     sars-cov-2    0.104698         0.440985
2       covid-19    0.063765         0.268576
3          virus    0.038658         0.162828
4          wuhan    0.024201         0.101936
5         severe    0.022264         0.093777
6          china    0.020190         0.085042
7       covid‐19    0.016923         0.071278
8        disease    0.016070         0.067687
9      infection    0.012474         0.052542
10      pandemic    0.011452         0.048236
11         viral    0.010274         0.043274
12   respiratory    0.009626         0.040543
13  transmission    0.004935         0.020786
14          cell    0.004646         0.019571
15             •    0.004439         0.018698
16         paper    0.003855         0.016239
17       patient    0.003689         0.015539
18      clinical    0.003582         0.015087
19      outbreak    0.003384         0.014255


In [17]:
df_word_imp.to_csv("global_word_importance_bow_surrogate.csv")

In [18]:
df_word_imp

word    importance  importance_norm
0      coronavirus  2.374177e-01     1.000000e+00
1       sars-cov-2  1.046977e-01     4.409855e-01
2         covid-19  6.376465e-02     2.685758e-01
3            virus  3.865818e-02     1.628277e-01
4            wuhan  2.420134e-02     1.019357e-01
...            ...           ...              ...
3405  hierarchical  9.365454e-13     3.822934e-12
3406    solidarity  9.002896e-13     3.670225e-12
3407    conducting  7.704694e-14     2.027379e-13
3408     organized  4.122099e-14     5.183944e-14
3409         spent  2.891339e-14     0.000000e+00

[3410 rows x 3 columns]

In [ ]:
from sklearn.inspection import permutation_importance
import numpy as np
import pandas as pd

# 1) Převod na hustou matici (pokud paměť dovolí)
X_dense = X_bow_test.toarray()

# 2) Rychlejší permutační importances (20 % řádků, 5 opakování)
perm = permutation_importance(
    surrogate_rf,
    X_dense,
    y_sur_test,
    n_repeats=5,
    random_state=42,
    n_jobs=-1,
    max_samples=0.2
)

words    = vect.get_feature_names_out()
imp_mean = perm.importances_mean     # průměrná změna R² při permutaci

# 3) Frekvence slov v dokumentech
freq = (X_dense > 0).sum(axis=0)

# 4) Average global surrogate importance
df_surr = pd.DataFrame({
    "word":    words,
    "imp_raw": imp_mean,
    "freq":    freq
})
df_surr = df_surr.query("freq > 0").copy()
df_surr["imp_avg"] = df_surr["imp_raw"] / df_surr["freq"]

df_surr = df_surr.sort_values("imp_avg", ascending=False).reset_index(drop=True)
print(df_surr.head(20))

# 5) Ulož výsledky do CSV
df_surr.to_csv("global_word_importance_bow_surrogate_v2.csv", index=False)
